# **User-User Filtering**

In [2]:
pip install pandas


   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.5 MB 2.6 MB/s eta 0:00:05
    --------------------------------------- 0.2/11.5 MB 3.1 MB/s eta 0:00:04
   - -------------------------------------- 0.4/11.5 MB 3.4 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/11.5 MB 3.7 MB/s eta 0:00:03
   -- ------------------------------------- 0.8/11.5 MB 3.5 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/11.5 MB 3.6 MB/s eta 0:00:03
   --- ------------------------------------ 1.1/11.5 MB 3.6 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/11.5 MB 3.6 MB/s eta 0:00:03
   ----- ---------------------------------- 1.5/11.5 MB 3.7 MB/s eta 0:00:03
   ----- ---------------------------------- 1.7/11.5 MB 3.7 MB/s eta 0:00:03
   ------ --------------------------------- 1.9/11.5 MB 3.7 MB/s eta 0:00:03
   ------- -------------------------------- 2.0/11.5 MB 3.7 MB/s eta 0:00:03
   --

In [3]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pylab import rcParams
# movies = pd.read_csv("/content/drive/My Drive/movielen/movies.csv", header=0)
# ratings = pd.read_csv("/content/drive/My Drive/movielen/ratings.csv", header=0)
movies = pd.read_csv('movies.csv', header=0)
ratings = pd.read_csv('ratings.csv', header=0)
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [2]:
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')
movies['title'] = movies['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)
movies.drop(columns=['genres'], inplace=True) #droping the column

movies.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [3]:
ratings.drop(columns=['timestamp'], inplace=True) #droping column
ratings.head()

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


In [4]:
# getting movie and their ratings
user = [
            {'title':'Breakfast Club, The', 'rating':4},
            {'title':'Toy Story', 'rating':2.5},
            {'title':'Jumanji', 'rating':3},
            {'title':"Pulp Fiction", 'rating':4.5},
            {'title':'Akira', 'rating':5}
         ]
inputMovie = pd.DataFrame(user)
inputMovie

,title,rating
0,"Breakfast Club, The",4.0
1,Toy Story,2.5
2,Jumanji,3.0
3,Pulp Fiction,4.5
4,Akira,5.0


In [5]:
Id = movies[movies['title'].isin(inputMovie['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovie = pd.merge(Id, inputMovie)
#Dropping information we won't use from the input dataframe
# inputMovie.drop('year', 1)
inputMovie.drop(columns=['year'], inplace=True)
inputMovie

,movieId,title,rating
0,1,Toy Story,2.5
1,2,Jumanji,3.0
2,296,Pulp Fiction,4.5
3,1274,Akira,5.0
4,1968,"Breakfast Club, The",4.0
5,164600,Akira,5.0


In [6]:
#Filtering out users that have watched movies that the input has watched and storing it
users = ratings[ratings['movieId'].isin(inputMovie['movieId'].tolist())]
users.head()

,userId,movieId,rating
0,1,296,5.0
70,2,1,3.5
141,2,1968,1.0
254,3,1,4.0
264,3,296,5.0


In [7]:
users.shape

(194464, 3)

In [8]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
#each group consists of all rows with the same value in the 'userId' column.
userSubsetGroup = users.groupby(['userId'])

In [9]:
#showing one such group example by getting all the users of a particular uderId
#retrieves one of the groups from the userSubsetGroup based on the specified userId (in this case, userId 1)
userSubsetGroup.get_group(1)

C:\Users\krgauravp\AppData\Local\Temp\ipykernel_14472\1541534571.py:3: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  userSubsetGroup.get_group(1)


,userId,movieId,rating
0,1,296,5.0


In [10]:
#Sorting it so users with movie most in common with the input will have priority
#sorts the userSubsetGroup by the number of items in each group in descending order.
# prioritize users who have rated the most movies in common with the input user
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [11]:
#retrieves the first three elements (groups) of the sorted userSubsetGroup, presumably to focus on the top three users with the most movies rated in common with the input user.
userSubsetGroup[0:3]

[((43,),
        userId  movieId  rating
  5858      43        1     4.0
  5859      43        2     3.5
  5884      43      296     5.0
  5974      43     1274     4.5
  6018      43     1968     4.5),
 ((171,),
         userId  movieId  rating
  20794     171        1     4.5
  20795     171        2     4.0
  20833     171      296     5.0
  20973     171     1274     4.0
  21047     171     1968     4.0),
 ((440,),
         userId  movieId  rating
  54640     440        1     3.5
  54641     440        2     2.0
  54669     440      296     5.0
  54756     440     1274     5.0
  54798     440     1968     4.5)]

In [ ]:
userSubsetGroup = userSubsetGroup[0:100]

In [18]:
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorDict = {} #empty dictionary 
#Pearson correlation coefficient between an input user and each user group in a user subset

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId') #Sorts the current user group by 'movieId'.
    inputMovie = inputMovie.sort_values(by='movieId') # Sorts the input movie DataFrame by 'movieId'.
    #Get the N for the formula
    n = len(group) #Gets the number of common movies between the input user and the current user group.
    #Get the review scores for the movies that they both have in common
    #Selects the rows from the input movie DataFrame where the 'movieId' matches with the current user group's movie IDs.
    temp = inputMovie[inputMovie['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    
    tempRatingList = temp['rating'].tolist()
    #put the current user group reviews in a list format
    #Stores the ratings of the current user group for common movies in a list.
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(n)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(n)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(n)

    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorDict[name] = 0

In [19]:
pearsonCorDict.items()

dict_items([((43,), 0.7613188819880644), ((171,), 0.026958193300858132), ((440,), 0.8038925870986305), ((597,), 0.0864585626556345), ((695,), 0.4313310928137531), ((757,), 0.835703992326648), ((772,), -0.017972128867238715), ((860,), 0.6827688932004732), ((939,), 0.39620290784653284), ((1203,), -0.018630665602665512), ((1242,), 0.46107317554294897), ((1401,), -0.14794534429106385), ((1652,), -0.1760901812651271), ((1894,), 0.444102681159703), ((1977,), -0.6105716541821835), ((2345,), -0.07188851546896004), ((2429,), 0.3774147062120368), ((2469,), 0.28614654455582733), ((2640,), -0.4068496968004207), ((2766,), -0.700913025822351), ((2982,), 0.09457559848219182), ((3150,), 0.17049858486761843), ((3266,), 0.37741470621203427), ((3272,), 0.6896865432884035), ((3274,), -0.26413527189768443), ((3624,), 0.14118624160050217), ((3660,), 0.9097992698698094), ((3760,), 0.057639041770424365), ((4006,), -0.26413527189768443), ((4019,), -0.0325128044381159), ((4060,), 0.4928713255803919), ((4246,), 

In [20]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.761319,"(43,)"
1,0.026958,"(171,)"
2,0.803893,"(440,)"
3,0.086459,"(597,)"
4,0.431331,"(695,)"


In [26]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers['userId'] = topUsers['userId'].apply(lambda x: x[0])
topUsers.head()

,similarityIndex,userId
83,0.970495,10115
48,0.951044,6272
76,0.924473,9678
31,0.924473,4246
26,0.909799,3660


In [30]:
# topUsersRating=topUsers.concat(ratings, left_on='userId', right_on='userId', how='inner')
# topUsersRating = pd.concat([topUsers, ratings], axis=1)
topUsersRating = pd.merge(topUsers, ratings, on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,0.970495,10115,1,2.0
1,0.970495,10115,2,2.5
2,0.970495,10115,3,2.5
3,0.970495,10115,6,3.5
4,0.970495,10115,10,3.0


In [31]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.970495,10115,1,2.0,1.940990
1,0.970495,10115,2,2.5,2.426237
2,0.970495,10115,3,2.5,2.426237
3,0.970495,10115,6,3.5,3.396732
4,0.970495,10115,10,3.0,2.911485


In [32]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,30.649868,105.709316
2,30.649868,84.694423
3,5.168702,14.334375
4,0.511496,1.022992
5,6.395879,12.636669


In [33]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.448932,1
2,2.763288,2
3,2.773303,3
4,2.000000,4
5,1.975752,5


In [34]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
7215,5.0,7215
111778,5.0,111778
171495,5.0,171495
26231,5.0,26231
6257,5.0,6257
831,5.0,831
26094,5.0,26094
26090,5.0,26090
213,5.0,213


In [35]:
movies.loc[movies['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
211,213,Burnt by the Sun (Utomlyonnye solntsem),1994
813,831,Stonewall,1995
6145,6257,I Am Curious (Yellow) (Jag är nyfiken - en fil...,1967
7090,7215,To Have and Have Not,1944
8513,26002,Confidential Report,1955
8570,26090,"Easy Life, The (Il Sorpasso)",1962
8572,26094,"Eclisse, L' (Eclipse)",1962
8674,26231,Performance,1970
21696,111778,Tracks,2013
45973,171495,Cosmos,NaN


In [ ]:
import pandas as pd
import numpy as np
from math import sqrt

# Sample data for movies and ratings
movies = pd.read_csv('movies.csv', header=0)
ratings = pd.read_csv('ratings.csv', header=0)

movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')
movies['title'] = movies['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)
movies.drop(columns=['genres'], inplace=True)

ratings.drop(columns=['timestamp'], inplace=True)

def recommend_movies(input_movies):
    # Get movie IDs based on input titles
    movie_ids = movies[movies['title'].isin(input_movies)]['movieId'].tolist()
    # Filter ratings for movies in input
    users = ratings[ratings['movieId'].isin(movie_ids)]
    # Group by user ID
    user_subset_group = users.groupby('userId')
    # Sort user groups by the number of movies in common with input
    user_subset_group = sorted(user_subset_group, key=lambda x: len(x[1]), reverse=True)
    user_subset_group = user_subset_group[:100]

    pearson_cor_dict = {}
    # For every user group in our subset
    for name, group in user_subset_group:
        group = group.sort_values(by='movieId')
        input_movies_sorted = sorted(input_movies)
        temp = group[group['movieId'].isin(movie_ids)]
        temp_rating_list = temp['rating'].tolist()
        temp_group_list = temp['rating'].tolist()
        n = len(temp)
        sxx = sum([i**2 for i in temp_rating_list]) - pow(sum(temp_rating_list), 2) / float(n)
        syy = sum([i**2 for i in temp_group_list]) - pow(sum(temp_group_list), 2) / float(n)
        sxy = sum(i * j for i, j in zip(temp_rating_list, temp_group_list)) - sum(temp_rating_list) * sum(
            temp_group_list) / float(n)
        if sxx != 0 and syy != 0:
            pearson_cor_dict[name] = sxy / sqrt(sxx * syy)
        else:
            pearson_cor_dict[name] = 0

    # Create DataFrame from Pearson correlation dict
    pearson_df = pd.DataFrame.from_dict(pearson_cor_dict, orient='index')
    pearson_df.columns = ['similarityIndex']
    pearson_df['userId'] = pearson_df.index
    pearson_df.index = range(len(pearson_df))

    # Take top similar users
    top_users = pearson_df.sort_values(by='similarityIndex', ascending=False)[0:50]

    # Merge with ratings to get top user ratings
    top_users_rating = top_users.merge(ratings, left_on='userId', right_on='userId', how='inner')

    # Calculate weighted rating
    top_users_rating['weightedRating'] = top_users_rating['similarityIndex'] * top_users_rating['rating']

    # Aggregate ratings by movieId
    temp_top_users_rating = top_users_rating.groupby('movieId').agg({'similarityIndex': 'sum', 'weightedRating': 'sum'})
    temp_top_users_rating.columns = ['sum_similarityIndex', 'sum_weightedRating']

    # Create recommendation DataFrame
    recommendation_df = pd.DataFrame()
    recommendation_df['weighted_average_recommendation_score'] = temp_top_users_rating['sum_weightedRating'] / \
                                                                 temp_top_users_rating['sum_similarityIndex']
    recommendation_df['movieId'] = temp_top_users_rating.index

    # Sort recommendations by score
    recommendation_df = recommendation_df.sort_values(by='weighted_average_recommendation_score', ascending=False)

    # Get movie details for top recommendations
    top_recommendations = movies.loc[movies['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

    return top_recommendations

# Example usage
input_movies = []
input_ratings = []

num_movies = int(input("Enter the number of movies you've watched: "))
for i in range(num_movies):
    title = input("Enter the title of movie {} : ".format(i + 1))
    rating = float(input("Enter your rating for movie {} (out of 5): ".format(title)))
    input_movies.append(title)
    input_ratings.append(rating)

# Get recommendations
recommendations = recommend_movies(input_movies)

# Display recommendations
print("\nTop 10 Recommendations:")
print(recommendations[['movieId', 'title']])

Enter the number of movies you've watched: 5
Enter the title of movie 1 : Breakfast Club, The
Enter your rating for movie Breakfast Club, The (out of 5): 4
Enter the title of movie 2 : Toy Story
Enter your rating for movie Toy Story (out of 5): 2.5
Enter the title of movie 3 : Jumanji
Enter your rating for movie Jumanji (out of 5): 3
Enter the title of movie 4 : Pulp Fiction
Enter your rating for movie Pulp Fiction (out of 5): 4.5
Enter the title of movie 5 : Akira
Enter your rating for movie Akira (out of 5): 5

Top 10 Recommendations:
      movieId                                              title
1984     2073                                           Fandango
5116     5224                                           Promises
5820     5932                                   Burden of Dreams
5920     6032                                  Little Romance, A
6074     6186                                  Gods and Generals
6089     6201                                          Lady Jane
61

In [1]:
import pandas as pd
# import numpy as np
from math import sqrt
# import matplotlib.pyplot as plt


# Sample data for movies and ratings
movies = pd.read_csv('movies.csv', header=0)
ratings = pd.read_csv('ratings.csv', header=0)

movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')
movies['title'] = movies['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)
movies.drop(columns=['genres'], inplace=True)

ratings.drop(columns=['timestamp'], inplace=True)

def recommend_movies(input_movies):
    # Get movie IDs based on input titles
    movie_ids = movies[movies['title'].isin(input_movies)]['movieId'].tolist()
    # Filter ratings for movies in input
    users = ratings[ratings['movieId'].isin(movie_ids)]
    # Group by user ID
    user_subset_group = users.groupby('userId')
    # Sort user groups by the number of movies in common with input
    user_subset_group = sorted(user_subset_group, key=lambda x: len(x[1]), reverse=True)
    user_subset_group = user_subset_group[:100]

    pearson_cor_dict = {}
    # For every user group in our subset
    for name, group in user_subset_group:
        group = group.sort_values(by='movieId')
        input_movies_sorted = sorted(input_movies)
        temp = group[group['movieId'].isin(movie_ids)]
        temp_rating_list = temp['rating'].tolist()
        temp_group_list = temp['rating'].tolist()
        n = len(temp)
        sxx = sum([i**2 for i in temp_rating_list]) - pow(sum(temp_rating_list), 2) / float(n)
        syy = sum([i**2 for i in temp_group_list]) - pow(sum(temp_group_list), 2) / float(n)
        sxy = sum(i * j for i, j in zip(temp_rating_list, temp_group_list)) - sum(temp_rating_list) * sum(
            temp_group_list) / float(n)
        if sxx != 0 and syy != 0:
            pearson_cor_dict[name] = sxy / sqrt(sxx * syy)
        else:
            pearson_cor_dict[name] = 0

    # Create DataFrame from Pearson correlation dict
    pearson_df = pd.DataFrame.from_dict(pearson_cor_dict, orient='index')
    pearson_df.columns = ['similarityIndex']
    pearson_df['userId'] = pearson_df.index
    pearson_df.index = range(len(pearson_df))

    # Take top similar users
    top_users = pearson_df.sort_values(by='similarityIndex', ascending=False)[0:50]

    # Merge with ratings to get top user ratings
    top_users_rating = top_users.merge(ratings, left_on='userId', right_on='userId', how='inner')

    # Calculate weighted rating
    top_users_rating['weightedRating'] = top_users_rating['similarityIndex'] * top_users_rating['rating']

    # Aggregate ratings by movieId
    temp_top_users_rating = top_users_rating.groupby('movieId').agg({'similarityIndex': 'sum', 'weightedRating': 'sum'})
    temp_top_users_rating.columns = ['sum_similarityIndex', 'sum_weightedRating']

    # Create recommendation DataFrame
    recommendation_df = pd.DataFrame()
    recommendation_df['weighted_average_recommendation_score'] = temp_top_users_rating['sum_weightedRating'] / \
                                                                 temp_top_users_rating['sum_similarityIndex']
    recommendation_df['movieId'] = temp_top_users_rating.index

    # Sort recommendations by score
    recommendation_df = recommendation_df.sort_values(by='weighted_average_recommendation_score', ascending=False)

    # Get movie details for top recommendations
    top_recommendations = movies.loc[movies['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

    return top_recommendations

#Example usage
input_movies = []
input_ratings = []

num_movies = int(input("Enter the number of movies you've watched: "))
for i in range(num_movies):
    title = input("Enter the title of movie {} : ".format(i + 1))
    rating = float(input("Enter your rating for movie {} (out of 5): ".format(title)))
    input_movies.append(title)
    input_ratings.append(rating)

# Get recommendations
recommendations = recommend_movies(input_movies)

# Display recommendations
print("\nTop 10 Recommendations:")
print(recommendations[['movieId', 'title','year']])



Top 10 Recommendations:
       movieId                                              title  year
17618    92064                                         You and Me  1938
17630    92120                         Good Bye (Bé omid é didar)  2011
17665    92229                          Vanquished, The (I vinti)  1953
39169   156581                                          Stutterer  2015
39180   156603  In Search of Blind Joe Death: The Saga of John...  2013
49086   178027                             People That Are Not Me  2016
51689   183681  American Radical: The Trials of Norman Finkels...  2009
51707   183721                            My Dad Is 100 Years Old  2005
56692   194714                                 In the Intense Now  2017
56720   194777                        The House of Violent Desire  2017
